In [1]:
import pdb
import gym
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import reward as rw
import reward.utils as U

/home/quazar/anaconda3/envs/torchrl_up/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
ENV = 'Humanoid-v2'
LOG_DIR = 'logs/humanoid/paper-v0-1'
REPAR = True
REWARD_SCALE = 20.
CLIP_GRAD = float('inf')
GAMMA = 0.99
TARGET_UP_WEIGHT = 0.005
BATCH_SIZE = 256
MAX_STEPS = 40e6

In [3]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [4]:
env = rw.envs.GymEnv(ENV)
env = rw.envs.wrappers.ActionBound(env)
runner = rw.runners.SingleRunner(env)
batcher = rw.batchers.ReplayBatcher(
    runner=runner,
    batch_size=256,
    replay_buffer_maxlen=1e6,
    learning_freq=1,
    grad_steps_per_batch=1,
    transforms=[
#         rw.batchers.transforms.StateRunNorm(),
        rw.batchers.transforms.RewardConstScaler(REWARD_SCALE),
    ],
)

state_features = batcher.get_state_info().shape[0]
num_actions = batcher.get_action_info().shape[0]

Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
class PolicyNN(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_units=256,
                 activation=nn.ReLU, log_std_range=(-20, 2)):
        super().__init__()
        self.log_std_range = log_std_range
        
        layers = []
        layers += [nn.Linear(num_inputs, hidden_units), activation()]
        layers += [nn.Linear(hidden_units, hidden_units), activation()]
        self.layers = nn.Sequential(*layers)
        
        self.mean = nn.Linear(hidden_units, num_outputs)
        self.mean.weight.data.uniform_(-3e-3, 3e-3)
        self.mean.bias.data.uniform_(-3e-3, 3e-3)
        
        self.log_std = nn.Linear(hidden_units, num_outputs)
        self.log_std.weight.data.uniform_(-3e-3, 3e-3)
        self.log_std.bias.data.uniform_(-3e-3, 3e-3)
        
    def forward(self, x):
        x = self.layers(x)
        mean = self.mean(x)
        log_std = self.log_std(x).clamp(*self.log_std_range)
        return mean, log_std        

In [6]:
class ValueNN(nn.Module):
    def __init__(self, num_inputs, hidden_units=256, activation=nn.ReLU):
        super().__init__()
        
        layers = []
        layers += [nn.Linear(num_inputs, hidden_units), activation()]
        layers += [nn.Linear(hidden_units, hidden_units), activation()]
        final_layer = nn.Linear(hidden_units, 1)
        final_layer.weight.data.uniform_(-3e-3, 3e-3)
        final_layer.bias.data.uniform_(-3e-3, 3e-3)
        layers += [final_layer]
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x)

In [7]:
class QValueNN(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_units=256, activation=nn.ReLU):
        super().__init__()
        
        layers = []
        layers += [nn.Linear(num_inputs + num_actions, hidden_units), activation()]
        layers += [nn.Linear(hidden_units, hidden_units), activation()]
        final_layer = nn.Linear(hidden_units, 1)
        final_layer.weight.data.uniform_(-3e-3, 3e-3)
        final_layer.bias.data.uniform_(-3e-3, 3e-3)
        layers += [final_layer]
        
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        state, action = x
        x = torch.cat([state, action], dim=1)
        return self.layers(x)

In [8]:
class TanhNormalPolicy(rw.policy.BasePolicy):
    def create_dist(self, state):
        parameters = self.nn(state)
        mean, log_std = parameters
        return rw.distributions.TanhNormal(loc=mean, scale=log_std.exp())

    def get_action(self, state, step):
        dist = self.create_dist(state=state)
        action = U.to_np(dist.sample())
        assert not np.isnan(action).any()
        return action

In [9]:
p_nn = PolicyNN(num_inputs=state_features, num_outputs=num_actions).to(device)
v_nn = ValueNN(num_inputs=state_features).to(device)
v_nn_target = ValueNN(num_inputs=state_features).to(device).eval()
q1_nn = QValueNN(num_inputs=state_features, num_actions=num_actions).to(device)
q2_nn = QValueNN(num_inputs=state_features, num_actions=num_actions).to(device)

In [10]:
U.copy_weights(from_nn=v_nn, to_nn=v_nn_target, weight=1.)

In [11]:
policy = TanhNormalPolicy(nn=p_nn)

In [12]:
p_opt = torch.optim.Adam(p_nn.parameters(), lr=3e-4)
v_opt = torch.optim.Adam(v_nn.parameters(), lr=3e-4)
q1_opt = torch.optim.Adam(q1_nn.parameters(), lr=3e-4)
q2_opt = torch.optim.Adam(q2_nn.parameters(), lr=3e-4)

In [13]:
logger = U.Logger(LOG_DIR)

Writing logs to: logs/humanoid/paper-v0-1


In [14]:
batcher.populate(n=1000, get_action_fn=policy.get_action)

Populating Replay Buffer...


In [ ]:
for batch in batcher.get_batches(MAX_STEPS, policy.get_action):
    batch = batch.to_tensor().concat_batch()

    ##### Calculate losses ######
    q1_batch = q1_nn((batch.state_t, batch.action))
    q2_batch = q2_nn((batch.state_t, batch.action))
    v_batch = v_nn(batch.state_t)

    dist = policy.create_dist(batch.state_t)
    if REPAR:
        action, pre_tanh_action = dist.rsample_with_pre()
    else:
        action, pre_tanh_action = dist.sample_with_pre()
    log_prob = dist.log_prob_pre(pre_tanh_action).sum(-1, keepdim=True)

    # Q loss
    v_target_tp1 = v_nn_target(batch.state_tp1)
    q_value_tp1 = U.estimators.td_target(rewards=batch.reward, dones=batch.done,
                                         v_tp1=v_target_tp1, gamma=GAMMA)
    q1_loss = F.mse_loss(q1_batch, q_value_tp1.detach())
    q2_loss = F.mse_loss(q2_batch, q_value_tp1.detach())

    # V loss    
    q1_new_t = q1_nn((batch.state_t, action))
    q2_new_t = q2_nn((batch.state_t, action))
    q_new_t = torch.min(q1_new_t, q2_new_t)
    next_value = q_new_t - log_prob
    v_loss = F.mse_loss(v_batch, next_value.detach())

    # Policy loss    
    if REPAR:        
        p_loss = (log_prob - q_new_t).mean()
    else:
        next_log_prob = q_new_t - v_batch
        p_loss = (log_prob * (log_prob - next_log_prob).detach()).mean()
    # Policy regularization losses
    mean_loss = 1e-3 * dist.loc.pow(2).mean()
    log_std_loss = 1e-3 * dist.scale.log().pow(2).mean()
    pre_tanh_loss = 0 * pre_tanh_action.pow(2).sum(1).mean()
    # Combine all losses
    p_loss += mean_loss + log_std_loss + pre_tanh_loss

    ###### Optimize ######
    q1_opt.zero_grad()
    q1_loss.backward()
#     torch.nn.utils.clip_grad_norm_(q1_nn.parameters(), CLIP_GRAD)
    q1_grad = U.mean_grad(q1_nn)
    q1_opt.step()

    q2_opt.zero_grad()
    q2_loss.backward()
#     torch.nn.utils.clip_grad_norm_(q2_nn.parameters(), CLIP_GRAD)
    q2_grad = U.mean_grad(q2_nn)
    q2_opt.step()

    v_opt.zero_grad()
    v_loss.backward()
#     torch.nn.utils.clip_grad_norm_(v_nn.parameters(), CLIP_GRAD)
    v_grad = U.mean_grad(v_nn)
    v_opt.step()

    p_opt.zero_grad()
    p_loss.backward()
#     torch.nn.utils.clip_grad_norm_(p_nn.parameters(), CLIP_GRAD)
    p_grad = U.mean_grad(p_nn)
    p_opt.step()

    ###### Update target value network ######
    U.copy_weights(from_nn=v_nn, to_nn=v_nn_target, weight=TARGET_UP_WEIGHT)

    ###### Write logs ######
    if batcher.num_steps % 4000 == 0 and batcher.runner.rewards:
        batcher.write_logs(logger)    

        logger.add_log('policy/loss', p_loss)
        logger.add_log('v/loss', v_loss)
        logger.add_log('q1/loss', q1_loss)
        logger.add_log('q2/loss', q2_loss)

        logger.add_log('policy/grad', p_grad)
        logger.add_log('v/grad', v_grad)
        logger.add_log('q1/grad', q1_grad)
        logger.add_log('q2/grad', q2_grad)

        logger.add_histogram('policy/log_prob', log_prob)
        logger.add_histogram('policy/mean', dist.loc)
        logger.add_histogram('policy/std', dist.scale.exp())
        logger.add_histogram('v/value', v_batch)
        logger.add_histogram('q1/value', q1_batch)
        logger.add_histogram('q2/value', q2_batch)

        logger.log(step=batcher.num_steps)


--------------------------------------------------------------
Env/Reward/Episode (New)                            |   121.18
Env/Length/Episode (New)                            |    24.49
Env/Reward/Episode (Last 50)                        |   178.31
Env/Length/Episode (Last 50)                        |    34.74
policy/loss                                         | -1071.56
v/loss                                              |  3609.77
q1/loss                                             |  8477.33
q2/loss                                             |  8528.21
policy/grad                                         |     0.01
v/grad                                              |  -375.73
q1/grad                                             |   160.64
q2/grad                                             |    48.89
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            |


--------------------------------------------------------------
Env/Reward/Episode (New)                            |   379.81
Env/Length/Episode (New)                            |    78.71
Env/Reward/Episode (Last 50)                        |   376.54
Env/Length/Episode (Last 50)                        |    78.08
policy/loss                                         | -2135.46
v/loss                                              |  7274.07
q1/loss                                             |  7576.76
q2/loss                                             |  9372.70
policy/grad                                         |    -0.01
v/grad                                              |  1775.26
q1/grad                                             |    97.51
q2/grad                                             |  -486.52
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            |


--------------------------------------------------------------
Env/Reward/Episode (New)                            |   437.22
Env/Length/Episode (New)                            |    88.09
Env/Reward/Episode (Last 50)                        |   440.59
Env/Length/Episode (Last 50)                        |    88.56
policy/loss                                         | -2538.03
v/loss                                              |  3858.49
q1/loss                                             | 12747.73
q2/loss                                             | 13180.04
policy/grad                                         |     0.01
v/grad                                              |  -270.66
q1/grad                                             |  -389.09
q2/grad                                             |  -182.70
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            |


--------------------------------------------------------------
Env/Reward/Episode (New)                            |   547.01
Env/Length/Episode (New)                            |   110.11
Env/Reward/Episode (Last 50)                        |   530.44
Env/Length/Episode (Last 50)                        |   106.32
policy/loss                                         | -3025.18
v/loss                                              |  3673.85
q1/loss                                             |  7223.64
q2/loss                                             |  7452.43
policy/grad                                         |    -0.02
v/grad                                              |   261.32
q1/grad                                             |   748.79
q2/grad                                             |   395.08
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            |


--------------------------------------------------------------
Env/Reward/Episode (New)                            |   472.94
Env/Length/Episode (New)                            |    98.44
Env/Reward/Episode (Last 50)                        |   477.25
Env/Length/Episode (Last 50)                        |    98.32
policy/loss                                         | -3178.41
v/loss                                              |  4266.91
q1/loss                                             |  6185.70
q2/loss                                             |  6122.18
policy/grad                                         |    -0.00
v/grad                                              |   921.86
q1/grad                                             |   -67.88
q2/grad                                             |   -75.42
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            |

...


--------------------------------------------------------------
Env/Reward/Episode (New)                            |  4758.74
Env/Length/Episode (New)                            |   825.40
Env/Reward/Episode (Last 50)                        |  5114.03
Env/Length/Episode (Last 50)                        |   889.72
policy/loss                                         | -7998.02
v/loss                                              |  5189.73
q1/loss                                             | 20971.95
q2/loss                                             | 21851.54
policy/grad                                         |     0.04
v/grad                                              |  1596.01
q1/grad                                             | -1215.09
q2/grad                                             | -1129.13
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            |


--------------------------------------------------------------
Env/Reward/Episode (New)                            |  5818.78
Env/Length/Episode (New)                            |  1000.00
Env/Reward/Episode (Last 50)                        |  5302.41
Env/Length/Episode (Last 50)                        |   919.10
policy/loss                                         | -8116.57
v/loss                                              | 11948.73
q1/loss                                             |  8101.26
q2/loss                                             | 13993.79
policy/grad                                         |    -0.00
v/grad                                              | -1845.24
q1/grad                                             |   305.43
q2/grad                                             |   508.23
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                          |  


--------------------------------------------------------------
Env/Reward/Episode (New)                            |  5206.27
Env/Length/Episode (New)                            |   898.80
Env/Reward/Episode (Last 50)                        |  5331.57
Env/Length/Episode (Last 50)                        |   919.04
policy/loss                                         | -8166.26
v/loss                                              |  8062.24
q1/loss                                             | 11660.27
q2/loss                                             |  6286.27
policy/grad                                         |     0.02
v/grad                                              | -1443.33
q1/grad                                             |   344.43
q2/grad                                             |   219.62
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                           | 


--------------------------------------------------------------
Env/Reward/Episode (New)                            |  4614.19
Env/Length/Episode (New)                            |   791.60
Env/Reward/Episode (Last 50)                        |  4916.89
Env/Length/Episode (Last 50)                        |   852.10
policy/loss                                         | -8197.66
v/loss                                              |  8437.56
q1/loss                                             | 14729.13
q2/loss                                             |  8293.15
policy/grad                                         |     0.03
v/grad                                              |  1262.92
q1/grad                                             |  -805.08
q2/grad                                             | -1196.58
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            |


--------------------------------------------------------------
Env/Reward/Episode (New)                            |  4411.86
Env/Length/Episode (New)                            |   771.20
Env/Reward/Episode (Last 50)                        |  4885.32
Env/Length/Episode (Last 50)                        |   846.98
policy/loss                                         | -8003.99
v/loss                                              |  4958.33
q1/loss                                             | 17207.22
q2/loss                                             | 10595.80
policy/grad                                         |    -0.00
v/grad                                              |   216.82
q1/grad                                             |  -255.74
q2/grad                                             |  -279.51
--------------------------------------------------------------

--------------------------------------------------------------
Env/Reward/Episode (New)                            |